In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
#import pycountry

PATH = r'../Horse_Racing_Data/CSV_Schema/'

# ISO 2 = alpha_2='AU'
# ISO 3 = alpha_3='AUS'

#NZ NZL

pd.options.display.max_colwidth = 1000000000

horses_data =  pd.read_csv(PATH + 'horses.csv')
#horses_data =  pd.read_csv('horses.csv')
horses_data.head()

num_horses = len(horses_data)
my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",  "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8"}


#horses_data['Country'].unique().tolist()

lookup_columns = ['Horse_ID','Horse_Name','Country','Age']
horses_lookup = horses_data[lookup_columns]
#horses_lookup.insert(loc=3, column="URL_Suffix", value=horses_data['Horse_Name'].str.replace(" ", "-").str.lower())
horses_lookup.insert(loc=4, column="URL", value="https://www.racenet.com.au/profiles/horse/"+horses_data['Horse_Name'].str.replace(" ", "-").str.replace("'", "").str.lower())
#horses_lookup.insert(loc=5, column="URL_Country_Suffix", value=horses_data['Horse_Name'].str.replace(" ", "-").str.lower()+"-"+horses_data['Country'].str.lower())
horses_lookup.insert(loc=5, column="URL_Country", value="https://www.racenet.com.au/profiles/horse/"+horses_data['Horse_Name'].str.replace(" ", "-").str.replace("'", "").str.lower()+"-"+horses_data['Country'].str.lower())
horses_lookup.insert(loc=6, column="URL_bn", value="https://www.breednet.com.au/horse/"+horses_data['Horse_Name'].str.replace(" ", "-").str.lower())
horses_lookup

horses_lookup.head()

,Horse_ID,Horse_Name,Country,Age,URL,URL_Country,URL_bn
0,1,SUPER AXIOM,AUS,3.0,https://www.racenet.com.au/profiles/horse/super-axiom,https://www.racenet.com.au/profiles/horse/super-axiom-aus,https://www.breednet.com.au/horse/super-axiom
1,2,SEA ELITE,AUS,4.0,https://www.racenet.com.au/profiles/horse/sea-elite,https://www.racenet.com.au/profiles/horse/sea-elite-aus,https://www.breednet.com.au/horse/sea-elite
2,3,LITTLE PLAYER,NZ,4.0,https://www.racenet.com.au/profiles/horse/little-player,https://www.racenet.com.au/profiles/horse/little-player-nz,https://www.breednet.com.au/horse/little-player
3,4,NEW FUTURE,AUS,4.0,https://www.racenet.com.au/profiles/horse/new-future,https://www.racenet.com.au/profiles/horse/new-future-aus,https://www.breednet.com.au/horse/new-future
4,5,RED MAJESTY,NZ,4.0,https://www.racenet.com.au/profiles/horse/red-majesty,https://www.racenet.com.au/profiles/horse/red-majesty-nz,https://www.breednet.com.au/horse/red-majesty


In [2]:
for i, row in horses_lookup.iterrows():
    url_ctry = row['URL_Country']
    html_content_ctry = requests.get(url_ctry, headers = my_headers).text
    soup_ctry = BeautifulSoup(html_content_ctry, 'lxml')
    result_list_ctry = len(soup_ctry.find_all('div',{'class':'profile-info__item--value'}))
    if result_list_ctry >= 5:
        foal_date = soup_ctry.find_all('div',{'class':'profile-info__item--value'})[5].find_all('span')[0].contents
        # if foal_date not found or listed as Invalid, then begin next iteration with other link
        if not foal_date:
            continue
        elif foal_date[0] == 'Invalid Date':
            continue
        horses_lookup.at[i, 'Foal_Date'] = str(foal_date).replace('[', '').replace(']', '').replace("'", "")
        horses_lookup.at[i, 'success_flag'] = 1
        print(row['Horse_Name']+': Success')
    else:
        url = row['URL']
        html_content = requests.get(url, headers = my_headers).text
        soup = BeautifulSoup(html_content, 'lxml')
        result_list = len(soup.find_all('div',{'class':'profile-info__item--value'}))
        if result_list >= 5:
            foal_date = soup.find_all('div',{'class':'profile-info__item--value'})[5].find_all('span')[0].contents
            # if foal_date not found or listed as Invalid, then begin next iteration with other link
            if not foal_date:
                continue
            elif foal_date[0] == 'Invalid Date':
                continue
            horses_lookup.at[i, 'Foal_Date'] = str(foal_date).replace('[', '').replace(']', '').replace("'", "")
            horses_lookup.at[i, 'success_flag'] = 1
            print(row['Horse_Name']+': Success')
        else:
            bn_url = row['URL_bn']
            html_content = requests.get(bn_url, headers = my_headers).text
            bn_soup = BeautifulSoup(html_content, 'lxml')
            bn_result_list = len(bn_soup.find_all('div',{'class':'horse-profile-row'}))
            if bn_result_list > 0:
                foal_date = bn_soup.find_all('div',{'class':'horse-profile-row'})[1].contents
                if not foal_date:
                    continue #if foal_date not found, then begin next iteration with other link
                foal_date = str(foal_date).replace("Foaled ", "").replace('[', '').replace(']', '').replace("'", "")
                foal_date = datetime.datetime.strptime(foal_date, '%B %d, %Y').strftime('%d/%m/%Y')
                horses_lookup.at[i, 'Foal_Date'] = foal_date
                horses_lookup.at[i, 'success_flag'] = 1
                print(row['Horse_Name']+': Success')          
            else:
                horses_lookup.at[i, 'success_flag'] = 0
                print(row['Horse_Name']+"\033[1m"+': Failed'+"\033[0m")

# check coverage of Foal Date after Web Scraping:
num_successes = horses_lookup['success_flag'].sum()
num_horses = len(horses_lookup)          
success_rate = (num_successes / num_horses)

print(success_rate)

horses_lookup

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\USER\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


SUPER AXIOM: Success
SEA ELITE: Success
LITTLE PLAYER: Success
NEW FUTURE: Success
RED MAJESTY: Success


KeyboardInterrupt: 